In [1]:
import pyodbc
import requests
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

Starting with data for 2016

Data set IRS 990 forms available via AWS:
 - https://aws.amazon.com/public-datasets/irs-990/

Gather EIN numbers for higher ed institutions in US via
 - https://nces.ed.gov/ipeds/Home/UseTheData

and select appropriate variables. CSV file "CSV_10282017-487/CSV_10282017-487.csv" is the lookup table


In [2]:
ein_dln_df = pd.read_csv("index_2016.csv")
univ_ein_name_df = pd.read_csv("CSV_10282017-487/CSV_10282017-487.csv")

In [3]:
ein_dln_df.head(1)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
0,13190365,EFILE,742661023,201412,02/09/2016,HARRIET AND HARMON KELLEY FOUNDATION FOR THE ARTS,990PF,93491315003445,201543159349100344


In [4]:
univ_ein_name_df.head(1)

,unitid,institution name,year,institution name.1,HD2016.Employer Identification Number
0,100654,Alabama A & M University,2016,Alabama A & M University,636001109


In [5]:
ein_to_univ = univ_ein_name_df.rename(columns={'HD2016.Employer Identification Number': 'EIN'})

In [6]:
univ_dln = pd.merge(ein_dln_df,ein_to_univ,on="EIN")  # only get 1105 (others are for-profit)

In [7]:
univ_dln.count()  # a DF with DLN for 2073 higher-ed insitutions in the US

RETURN_ID             1105
FILING_TYPE           1105
EIN                   1105
TAX_PERIOD            1105
SUB_DATE              1105
TAXPAYER_NAME         1105
RETURN_TYPE           1105
DLN                   1105
OBJECT_ID             1105
unitid                1105
institution name      1105
year                  1105
institution name.1    1105
dtype: int64

In [8]:
xml_to_get = univ_dln.OBJECT_ID

In [9]:
# This is a template for how to get a dataframe of officer compensation by university name:

def get_officers_by_name(name):
    """
    Get a dataframe of paid officers for the institution
    inputs:
        name -> (title case), e.g. Pacific University
                matches institution name field
    """
    if (univ_dln["institution name"] == name).any():
        dln = univ_dln[univ_dln["institution name"] == name]
    elif (univ_dln["institution name"].str.contains(name)).any():
        dln = univ_dln[univ_dln["institution name"].str.contains(name)][0]
    else:
        print("Not Found")
        return []

    ob_id = dln.OBJECT_ID.iloc[0]

    url = "https://s3.amazonaws.com/irs-form-990/" + str(ob_id) + "_public.xml"
    response = requests.get(url)
    root = ET.fromstring(response.content)

    paid_officers = root.findall(".//{http://www.irs.gov/efile}Form990PartVIISectionAGrp")

    df_list = []

    for i,officer in enumerate(paid_officers):  # enumerate so we can index properly
        #print(i[4].text)
        keys = []
        vals = []
        for field in officer:
            #print(field.text)
            keys.append(field.tag[26:])
            vals.append(field.text)

        #print(dict(zip(keys,vals)))
        row = pd.DataFrame(dict(zip(keys,vals)),index=[i])  # have to index with a single item
        df_list.append( row )
        #print(row)
        #df.append(row)

    officers_df = pd.concat(df_list)

    officers_df["Institution"] = name
    
    return officers_df

In [10]:
get_officers_by_name("Pacific University")

,AverageHoursPerWeekRltdOrgRt,AverageHoursPerWeekRt,HighestCompensatedEmployeeInd,IndividualTrusteeOrDirectorInd,OfficerInd,OtherCompensationAmt,PersonNm,ReportableCompFromOrgAmt,ReportableCompFromRltdOrgAmt,TitleTxt,Institution
0,0.0,40.0,NaN,X,X,34288,Lesley M Hallick,387804,0,President/Trustee,Pacific University
1,0.0,40.0,NaN,X,NaN,19869,Michelle M Cowing,119951,0,Professor/Trustee,Pacific University
2,0.0,40.0,NaN,X,NaN,6371,Catherine E Kim,70754,0,Professor/Trustee,Pacific University
3,0.0,1.0,NaN,X,NaN,0,Kimberly Bittner,0,0,Student/Trustee,Pacific University
4,0.0,1.0,NaN,X,NaN,0,Kailea Saplan,0,0,Student/Trustee,Pacific University
5,0.0,1.0,NaN,X,NaN,0,Julie Berglund Baker,0,0,Trustee,Pacific University
6,0.0,1.0,NaN,X,NaN,0,Robert O Barrett,0,0,Trustee,Pacific University
7,0.0,1.0,NaN,X,NaN,0,Evona M Brim,0,0,Trustee,Pacific University
8,0.0,1.0,NaN,X,NaN,0,Mindy Cameron,0,0,Trustee,Pacific University
9,0.0,1.0,NaN,X,NaN,0,Dori M Carlson,0,0,Trustee,Pacific University


## TODO: pull for all and merge into one master df

In [11]:
# This is a template for how to get a dataframe of officer compensation by university name:

def get_officers_by_obid(obid):
    if (univ_dln.OBJECT_ID == obid).any():
        dln = univ_dln[univ_dln.OBJECT_ID == obid]
    else:
        print("Not Found")
        return []

    ob_id = dln.OBJECT_ID.iloc[0]

    url = "https://s3.amazonaws.com/irs-form-990/" + str(ob_id) + "_public.xml"
    response = requests.get(url)
    root = ET.fromstring(response.content)

    paid_officers = root.findall(".//{http://www.irs.gov/efile}Form990PartVIISectionAGrp")

    df_list = []

    for i,officer in enumerate(paid_officers):  # enumerate so we can index properly
        #print(i[4].text)
        keys = []
        vals = []
        for field in officer:
            #print(field.text)
            keys.append(field.tag[26:])
            vals.append(field.text)

        #print(dict(zip(keys,vals)))
        row = pd.DataFrame(dict(zip(keys,vals)),index=[i])  # have to index with a single item
        df_list.append( row )
        #print(row)
        #df.append(row)
        
    if len(df_list) > 0:
        officers_df = pd.concat(df_list)
    else:
        return pd.DataFrame([])

    name = univ_dln[univ_dln.OBJECT_ID == obid]['institution name'].iloc[0]
    officers_df["Institution"] = name
    
    return officers_df

In [14]:
%%time
df_list = []
for obid in xml_to_get:
    try:
        df = get_officers_by_obid(obid)
    except ConnectionError:
        pass
    if not df.empty:
        df_list.append(df)

CPU times: user 1min 22s, sys: 4.77 s, total: 1min 27s
Wall time: 15min 22s


In [15]:
len(df_list)

1101

In [16]:
total = pd.concat(df_list)

In [17]:
total.columns

Index(['AverageHoursPerWeekRltdOrgRt', 'AverageHoursPerWeekRt', 'BusinessName',
       'FormerOfcrDirectorTrusteeInd', 'HighestCompensatedEmployeeInd',
       'IndividualTrusteeOrDirectorInd', 'Institution',
       'InstitutionalTrusteeInd', 'KeyEmployeeInd', 'OfficerInd',
       'OtherCompensationAmt', 'PersonNm', 'ReportableCompFromOrgAmt',
       'ReportableCompFromRltdOrgAmt', 'TitleTxt'],
      dtype='object')

### Convert numeric columns to int for proper aggregation.
`df[['col2','col3']] = df[['col2','col3']].apply(pd.to_numeric)`

In [18]:
total['ReportableCompFromOrgAmt'] = total['ReportableCompFromOrgAmt'].apply(pd.to_numeric)

In [19]:
# Find max compensation grouped by institution:
total.groupby('Institution')['ReportableCompFromOrgAmt'].max()

Institution
Abilene Christian University                       263420.0
Adelphi University                                 553588.0
Adrian College                                     729149.0
Adventist University of Health Sciences                 0.0
Agnes Scott College                                267503.0
Alaska Pacific University                          220360.0
Albany College of Pharmacy and Health Sciences     473588.0
Albertus Magnus College                            360889.0
Albion College                                     154239.0
Albright College                                   322348.0
Alderson Broaddus University                       170000.0
Alfred University                                  358977.0
Alice Lloyd College                                154293.0
Allegheny College                                  348833.0
Allen College                                           0.0
Allen University                                   245303.0
Alma College                

In [20]:
sys.getsizeof(total)

37213907

In [21]:
total.to_pickle("total1101.pkl")

In [22]:
!ls -lrath

total 118000
drwxr-xr-x  20 dawe7269  1399141610   680B Oct 28 02:43 ..
drwxr-xr-x   3 dawe7269  1399141610   102B Oct 28 10:22 .ipynb_checkpoints
-rw-r--r--@  1 dawe7269  1399141610    67K Oct 28 12:09 CSV_10282017-487.zip
drwx------@  4 dawe7269  1399141610   136B Oct 28 12:12 CSV_10282017-487
-rw-r--r--   1 dawe7269  1399141610    88K Oct 28 12:12 IPEDS Data Center Variable Selection.pdf
-rw-r--r--@  1 dawe7269  1399141610    43M Oct 28 12:53 index_2016.csv
-rw-r--r--@  1 dawe7269  1399141610   3.9M Oct 28 14:05 efiler_master_concordance.csv
-rw-r--r--@  1 dawe7269  1399141610   6.0K Oct 28 14:05 .DS_Store
-rw-r--r--   1 dawe7269  1399141610   213K Oct 28 22:25 total50.pkl
-rw-r--r--   1 dawe7269  1399141610   1.2M Oct 28 22:35 total250.pkl
-rw-r--r--   1 dawe7269  1399141610   3.1M Oct 29 11:24 total580.pkl
-rw-r--r--   1 dawe7269  1399141610    58K Oct 29 20:52 My_Tuition_at_Work.ipynb
-rw-r--r--   1 dawe7269  1399141610   6.2M Oct 29 20:52 total1101.pkl
drwxr-xr-x  14 dawe7269  1

In [23]:
schoolofficers = total.pivot_table(index='Institution', columns='TitleTxt', values='ReportableCompFromOrgAmt')

In [30]:
# Sorted list of top positions for a given college:

# need to show only nonnull so use dropna:
schoolofficers.loc["Whitman College"].dropna().sort_values(ascending=False)

TitleTxt
President                        372596.0
CFO/Treasurer                    220336.0
Faculty                          210134.6
VP for Development               209663.0
Provost & Dean of the Faculty    186056.0
Dean of Students                 171640.0
Chief Information Officer        157638.0
Vice Chair                            0.0
Trustee                               0.0
Chair                                 0.0
Name: Whitman College, dtype: float64